In [80]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# 01. file read, preprocessing

In [ ]:
# 01. file read, preprocessing

In [27]:
# 엑셀 파일 읽어오기
import pandas as pd
import numpy as np
df = pd.read_excel('crawling_data_all.xlsx')
df

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Document Type,Publication Stage,Open Access,Source,EID,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,"Jouin C., Mlocek Z.",57219668257;57219668269;,Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,115,3,NaN,61.0,70.0,...,Article,Final,NaN,Scopus,2-s2.0-85094632803,NaN,NaN,NaN,NaN,NaN
1,Buzási A.,56692966000;,Climate vulnerability and adaptation challenge...,2021,Climate,9,2,25,1.0,17.0,...,Article,Final,NaN,Scopus,2-s2.0-85100598191,NaN,NaN,NaN,NaN,NaN
2,"Yu G.-R., Yang M., Chen Z., Zhang L.-M.",55807480400;55703268000;57001631400;55077923500;,Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,32,4,NaN,1141.0,1153.0,...,Article,Final,NaN,Scopus,2-s2.0-85104538541,NaN,NaN,NaN,NaN,NaN
3,"Thorn J.P.R., Biancardi Aleu R., Wijesinghe A....",56613034600;57260808600;57226747891;5726048210...,Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,216,NaN,104235,NaN,NaN,...,Article,Final,NaN,Scopus,2-s2.0-85115004828,NaN,NaN,NaN,NaN,NaN
4,"Schuerch M., Spencer T., Temmerman S., Kirwan ...",54783506900;55156903100;6602902860;10044268300...,Future response of global coastal wetlands to ...,2018,Nature,561,7722,NaN,231.0,234.0,...,Article,Final,NaN,Scopus,2-s2.0-85053248555,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Antoušková M.,42161107100;,Economic value of recreation - Determinants in...,2012,Agris On-line Papers in Economics and Informatics,4,SUPPL.SPL. 4,NaN,3.0,9.0,...,Article,Final,NaN,Scopus,2-s2.0-84874044123,NaN,NaN,NaN,NaN,NaN
1052,Nicol R.,55173678200;,Entering the Fray: The role of outdoor educati...,2014,Educational Philosophy and Theory,46,5,NaN,449.0,461.0,...,Article,Final,NaN,Scopus,2-s2.0-84899996845,NaN,NaN,NaN,NaN,NaN
1053,"Boller F., Hunziker M., Conedera M., Elsasser ...",36647235100;6701797579;6603211787;7005909792;8...,Fascinating remoteness: The dilemma of hiking ...,2010,Mountain Research and Development,30,4,NaN,320.0,331.0,...,Article,Final,NaN,Scopus,2-s2.0-78649521966,NaN,NaN,NaN,NaN,NaN
1054,"Vedel S.E., Thorsen B.J., Jacobsen J.B.",15057304900;6701749452;35773326500;,"First-movers, non-movers, and social gains fro...",2009,Ecological Economics,68,2021-08-09 00:00:00,NaN,2363.0,2371.0,...,Article,Final,NaN,Scopus,2-s2.0-65649085241,NaN,NaN,NaN,NaN,NaN


In [28]:
# 초록이 없는 것 조회
df[df['Abstract'].isnull()]

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Document Type,Publication Stage,Open Access,Source,EID,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28


In [29]:
# 요약문의 형식을 살피기
df.Abstract[11]

'Inadequate sanitation infrastructure is a global problem that is particularly impacting rural areas. And decentralized wastewater management system is considered as the feasible solution for rural sewage treatment (RuST). However, determining the cost-effective (CE) pattern for decentralized RuST is methodologically challenging because of scarce decision-support tools. In this research, a RuST optimization model (RuST-OM) was developed to gain an insight into the CE pattern of RuST based on the greedy algorithm. This model involves tradeoffs in the economy-of-scale and technology of wastewater treatment system versus the cost and energy consumption of the sewage collection system. The investment associated with the CE pattern for RuST is closely linked to the environmental demand, RuST coverage, topographic complexity, and degree of household dispersion. The cost of the CE pattern falls between the onsite-B and community-based pattern, and this range represents the optimized interval 

In [30]:
# 요약문에서 실제로 내용이 있는 부분('@' 앞)만을 가져와, 새 컬럼 Abs_split에 저장
abs_split = df['Abstract'].str.split('©')
df['Abs_split'] = abs_split.str.get(0)
df.Abs_split[11]

'Inadequate sanitation infrastructure is a global problem that is particularly impacting rural areas. And decentralized wastewater management system is considered as the feasible solution for rural sewage treatment (RuST). However, determining the cost-effective (CE) pattern for decentralized RuST is methodologically challenging because of scarce decision-support tools. In this research, a RuST optimization model (RuST-OM) was developed to gain an insight into the CE pattern of RuST based on the greedy algorithm. This model involves tradeoffs in the economy-of-scale and technology of wastewater treatment system versus the cost and energy consumption of the sewage collection system. The investment associated with the CE pattern for RuST is closely linked to the environmental demand, RuST coverage, topographic complexity, and degree of household dispersion. The cost of the CE pattern falls between the onsite-B and community-based pattern, and this range represents the optimized interval 

In [31]:
#초록과 저자 키워드 있는 것  -> 새로운 컬럼 abs+key에 합치기 
df["abs+key"]= df["Abs_split"] +" "+df["Index Keywords"]
    
##한슬언니한테 물어보기 

df["abs+key"] = df["abs+key"].fillna(df["Abs_split"])
df["abs+key"]

0       Stormwater management is one of the greatest c...
1       Wine producers face several challenges regardi...
2       Terrestrial ecosystems are the core components...
3       (Despite a growing recognition of the importan...
4       The response of coastal wetlands to sea-level ...
                              ...                        
1051    The presented article deals with an indirect e...
1052    This article draws on different bodies of know...
1053    Remote areas devoid of roads and tourist trans...
1054    The importance of ecosystem services for manki...
1055    Soil moisture content (0–90 cm depth) and nitr...
Name: abs+key, Length: 1056, dtype: object

In [39]:
#df_final 저장하기 'Unnamed:24', 'Unnamed:25','Unnamed:26','Unnamed:27','Unnamed:28'
# 필요없는 column drop
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_final= df.drop(['Volume',	'Issue',	'Art. No.',	'Page start',	'Page end',	'Page count','Cited by','DOI','Link','Author(s) ID','Abstract', 'Author Keywords', 'Index Keywords', 'Abs_split','Document Type','Open Access','EID',],axis=1)

In [40]:
df_final

,Authors,Title,Year,Source title,Affiliations,Authors with affiliations,Publication Stage,Source,abs+key
0,"Jouin C., Mlocek Z.",Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,"Atelier Ld, Paris, France","Jouin, C., Atelier Ld, Paris, France; Mlocek, ...",Final,Scopus,Stormwater management is one of the greatest c...
1,Buzási A.,Climate vulnerability and adaptation challenge...,2021,Climate,"Department of Environmental Economics, Budapes...","Buzási, A., Department of Environmental Econom...",Final,Scopus,Wine producers face several challenges regardi...
2,"Yu G.-R., Yang M., Chen Z., Zhang L.-M.",Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,Synthesis Research Center of Chinese Ecosystem...,"Yu, G.-R., Synthesis Research Center of Chines...",Final,Scopus,Terrestrial ecosystems are the core components...
3,"Thorn J.P.R., Biancardi Aleu R., Wijesinghe A....",Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,"African Climate and Development Initiative, Un...","Thorn, J.P.R., African Climate and Development...",Final,Scopus,(Despite a growing recognition of the importan...
4,"Schuerch M., Spencer T., Temmerman S., Kirwan ...",Future response of global coastal wetlands to ...,2018,Nature,"Lincoln Centre for Water and Planetary Health,...","Schuerch, M., Lincoln Centre for Water and Pla...",Final,Scopus,The response of coastal wetlands to sea-level ...
...,...,...,...,...,...,...,...,...,...
1051,Antoušková M.,Economic value of recreation - Determinants in...,2012,Agris On-line Papers in Economics and Informatics,"Faculty of Economics and Management, Czech Uni...","Antoušková, M., Faculty of Economics and Manag...",Final,Scopus,The presented article deals with an indirect e...
1052,Nicol R.,Entering the Fray: The role of outdoor educati...,2014,Educational Philosophy and Theory,"School of Education, University of Edinburgh, ...","Nicol, R., School of Education, University of ...",Final,Scopus,This article draws on different bodies of know...
1053,"Boller F., Hunziker M., Conedera M., Elsasser ...",Fascinating remoteness: The dilemma of hiking ...,2010,Mountain Research and Development,"Department of Geography, University of Zurich,...","Boller, F., Department of Geography, Universit...",Final,Scopus,Remote areas devoid of roads and tourist trans...
1054,"Vedel S.E., Thorsen B.J., Jacobsen J.B.","First-movers, non-movers, and social gains fro...",2009,Ecological Economics,"Forest and Landscape, University of Copenhagen...","Vedel, S.E., Forest and Landscape, University ...",Final,Scopus,The importance of ecosystem services for manki...


# 02. 영문 형태소 분석 using nltk

In [41]:
documents = list(df_final['abs+key'])
print(len(documents))
documents[0]

1056


"Stormwater management is one of the greatest challenges in the modern urban environment. Doing it well – in a way which respects the natural environment and responds to its constraints – allows to address a multitude of other issues: social (creating amenity space, rebuilding connections between people and nature and improving the living environment), economical (increasing the land value, reducing development and maintenance costs), environmental (ecological corridors, biodiversity, mitigating heath island effects)… Urban designers have got an array of tools to use in order to integrate natural water management into city projects. The aim of this article is, using a number of case study projects in Villeneuve-la-Garenne, to present some of the most commonly used, efficient and simple to implement solutions which can be used to integrate NBS (Nature Based Solutions) into the stormwater management of a city at an individual project scale. The case studies will also provide a starting p

## Preprocessing

In [105]:
#preprocessing 
import re 
import nltk 
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import preprocess_string

nltk.download('stopwords')
stop_words = stopwords.words('english')

def clean_text(d):
    pattern = r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', d)
    return d

def clean_stopword(d):
    return ' '.join([w.lower() for w in d.split() if w not in stop_words and len(w) > 3])

def preprocessing(d):
    return preprocess_string(d)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ra4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df_final['abs+key']=df['abs+key'].apply(clean_text)
df_final['abs+key']

0       Stormwater management is one of the greatest c...
1       Wine producers face several challenges regardi...
2       Terrestrial ecosystems are the core components...
3       (Despite a growing recognition of the importan...
4       The response of coastal wetlands to sea-level ...
                              ...                        
1051    The presented article deals with an indirect e...
1052    This article draws on different bodies of know...
1053    Remote areas devoid of roads and tourist trans...
1054    The importance of ecosystem services for manki...
1055    Soil moisture content (0–90 cm depth) and nitr...
Name: abs+key, Length: 1056, dtype: object

In [ ]:
df_final['abs+key']=df['abs+key'].apply(clean_stopword)
df_final['abs+key']

0       stormwater management greatest challenges mode...
1       wine producers face several challenges regardi...
2       terrestrial ecosystems core components biosphe...
3       (despite growing recognition importance design...
4       response coastal wetlands sea-level rise twent...
                              ...                        
1051    presented article deals indirect economic valu...
1052    this article draws different bodies knowledge ...
1053    remote areas devoid roads tourist transport in...
1054    importance ecosystem services mankind stresses...
1055    soil moisture content (0–90 depth) nitrate-nit...
Name: abs+key, Length: 1056, dtype: object

The original list : Stormwater management is one of the greatest challenges in the modern urban environment. Doing it well – in a way which respects the natural environment and responds to its constraints – allows to address a multitude of other issues: social (creating amenity space, rebuilding connections between people and nature and improving the living environment), economical (increasing the land value, reducing development and maintenance costs), environmental (ecological corridors, biodiversity, mitigating heath island effects)… Urban designers have got an array of tools to use in order to integrate natural water management into city projects. The aim of this article is, using a number of case study projects in Villeneuve-la-Garenne, to present some of the most commonly used, efficient and simple to implement solutions which can be used to integrate NBS (Nature Based Solutions) into the stormwater management of a city at an individual project scale. The case studies will also p

## Tokenization

In [71]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [72]:
#word_tokenize-> '기준으로 다 나눠버림 
#WordPunctTokenizer -> 구두점을 별도로 분류하는 특징 
#text_to_word_sequence() => 모든 알파벳을 소문자로 바꾸면서 마침표, 컴마, 느낌표 제거 아포스트로피  보존 


# initializing list
documents
  
# printing original list
print("The original list : " + str(documents[0]))
  
# using list comprehension + split()
# Tokenizing strings in list of strings
tokenized_abs = [text_to_word_sequence(sub) for sub in documents]
  
# print result
print("The list after split of strings is : " + str(tokenized_abs[0]))

##df_final에 tokenized된 abstract + key list 형태로 달기 
df_final['tokenized_abs'] = np.array(tokenized_abs)




print(df_final['abs+key'])
print(df_final['tokenized_abs'])

The original list : Stormwater management is one of the greatest challenges in the modern urban environment. Doing it well – in a way which respects the natural environment and responds to its constraints – allows to address a multitude of other issues: social (creating amenity space, rebuilding connections between people and nature and improving the living environment), economical (increasing the land value, reducing development and maintenance costs), environmental (ecological corridors, biodiversity, mitigating heath island effects)… Urban designers have got an array of tools to use in order to integrate natural water management into city projects. The aim of this article is, using a number of case study projects in Villeneuve-la-Garenne, to present some of the most commonly used, efficient and simple to implement solutions which can be used to integrate NBS (Nature Based Solutions) into the stormwater management of a city at an individual project scale. The case studies will also p

C:\Users\ra4\AppData\Local\Temp\ipykernel_8728\161732609.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_final['tokenized_abs'] = np.array(tokenized_abs)


# Gensim을 이용한 Topic Modeling

In [90]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_final['tokenized_abs'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df_final['tokenized_abs']], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[df_final['tokenized_abs'][0]]])
print("tokenized_abs개수",len(df_final['tokenized_abs'][0]))
print("trigam+mod 개수",len(trigram_mod[bigram_mod[df_final['tokenized_abs'][0]]]))

['stormwater', 'management', 'is', 'one', 'of', 'the', 'greatest', 'challenges', 'in', 'the', 'modern', 'urban', 'environment', 'doing', 'it', 'well', '–', 'in', 'a', 'way', 'which', 'respects', 'the', 'natural', 'environment', 'and', 'responds', 'to', 'its', 'constraints', '–', 'allows', 'to', 'address', 'a', 'multitude', 'of', 'other', 'issues', 'social', 'creating', 'amenity', 'space', 'rebuilding', 'connections', 'between', 'people', 'and', 'nature', 'and', 'improving', 'the', 'living', 'environment', 'economical', 'increasing', 'the', 'land', 'value', 'reducing', 'development', 'and', 'maintenance', 'costs', 'environmental', 'ecological', 'corridors', 'biodiversity', 'mitigating', 'heath', 'island', 'effects', '…', 'urban', 'designers', 'have', 'got', 'an', 'array', 'of', 'tools', 'to', 'use', 'in', 'order', 'to', 'integrate', 'natural', 'water', 'management', 'into', 'city', 'projects', 'the', 'aim', 'of', 'this', 'article', 'is', 'using', 'a', 'number', 'of', 'case_study', 'proj

In [93]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [106]:


# Remove Stop Words
data_words_nostops = remove_stopwords(df_final['tokenized_abs'])

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


print(data_lemmatized[:1])

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")